In [21]:
import zmq
from distributed_video.load_balancer.constants import RESP_SOCKET_URL
from distributed_video.load_balancer.node_manager import NodesDirectory

# context = zmq.Context()

# context1 = zmq.Context()
# context1.destroy()
# context1 = zmq.Context()
  
# # send work
# ack_socket = context1.socket(zmq.PUSH)
# ack_socket.close()
# ack_socket.bind(f"tcp://{RESP_SOCKET_URL}")

# nd = NodesDirectory()
# nd.close_all_sockets()
# nd.bind_all_sockets()
# nd.nodes[0].context.term()

context2 = zmq.Context()
consumer_sender = context2.socket(zmq.PUSH)
# try:
#     # consumer_sender.bind(f"tcp://{RESP_SOCKET_URL}")
#     raise zmq.error.ZMQBaseError()
# except zmq.error.ZMQError as ex:
#     print("============= Error with zmq")
#     print("socket already bounded")
#     print(ex)
#     # consumer_sender.close()
#     consumer_sender.connect(f"tcp://{RESP_SOCKET_URL}")
# consumer_sender.bind(f"tcp://{RESP_SOCKET_URL}")
consumer_sender.connect(f"tcp://{RESP_SOCKET_URL}")
print("socket binded")
# consumer_sender.send_json({"hello": 'world'})

Exception ignored in: <function Context.__del__ at 0x7fa486da4820>
Traceback (most recent call last):
  File "/home/k2/Work/SCU/Distributed Systems/distributed-video-processing/.venv/lib/python3.10/site-packages/zmq/sugar/context.py", line 151, in __del__
    self.destroy()
  File "/home/k2/Work/SCU/Distributed Systems/distributed-video-processing/.venv/lib/python3.10/site-packages/zmq/sugar/context.py", line 332, in destroy
    self.term()
  File "/home/k2/Work/SCU/Distributed Systems/distributed-video-processing/.venv/lib/python3.10/site-packages/zmq/sugar/context.py", line 274, in term
    super().term()
  File "zmq/backend/cython/context.pyx", line 83, in zmq.backend.cython.context.Context.term
  File "zmq/backend/cython/checkrc.pxd", line 13, in zmq.backend.cython.checkrc._check_rc
KeyboardInterrupt: 


socket binded


In [24]:
import requests

video_path = "/home/k2/Work/SCU/Distributed Systems/distributed-video-processing/distributed_video/assets/test-face.mp4"

files = {'file': open(video_path, 'rb')}

response = requests.post("http://127.0.0.1:6000/apply-filter", files=files, data={'file_name': 'sample-video.mp4'})
# response = requests.post("http://localhost:6000/test")

response.json()

{'task_id': 'baa67dc1-2f8f-4235-981d-50fa0feeb82b',
 'message': 'uploaded successfully'}

In [22]:
from distributed_video.libs.utils import BlobStore
import cv2
import numpy as np
import os

def aggregator(task_id, total_frames):
    img_array = []
    bs = BlobStore()
    # file_list = glob.glob("/home/k2/Work/SCU/Distributed Systems/distributed-video-processing/distributed_video/bucket/"+task_id+"/*.jpg")
    # print(file_list)    
    # for filename in file_list:
    for i in range(total_frames):
        data = bs.read_file(file_name=f'{i}.jpg', task="0c1a6e38-ad8d-476c-914e-b1aa00c97146")
        img = cv2.cvtColor(np.frombuffer(data, np.uint8), cv2.IMREAD_COLOR)
        cv2.imshow('test', img)
        height, width, layers = img.shape
        size = (width, height)
        img_array.append(img)

    print(len(img_array))
    save_path = os.path.join('/home/k2/Work/SCU/Distributed Systems/distributed-video-processing/distributed_video/load_balancer/media', str(task_id)+'.avi')
    out = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*"MJPG"), 15, size)

    for i in range(len(img_array)):
        out.write(img_array[i])
    out.release()
    return 'success'

aggregator('0c1a6e38-ad8d-476c-914e-b1aa00c97146', 24)

24


'success'

: 

In [6]:
image_path = '/home/k2/Pictures/vlcsnap-2023-02-21-18h44m20s534.png'

import cv2
from distributed_video.libs.utils import BlobStore
import os

bs = BlobStore()

image = cv2.imread(image_path)

os.makedirs(os.path.join('/home/k2/Work/SCU/Distributed Systems/distributed-video-processing/distributed_video/bucket', 'please'), exist_ok=True)
bs.write_frame('temp.jpg', 'please', image)

bs.read_frame('temp.jpg', 'please')

array([[[ 95, 117, 165],
        [ 95, 117, 165],
        [ 95, 117, 165],
        ...,
        [ 90, 113, 158],
        [ 90, 113, 158],
        [ 90, 113, 158]],

       [[ 95, 117, 165],
        [ 95, 117, 165],
        [ 95, 117, 165],
        ...,
        [ 90, 113, 158],
        [ 90, 113, 158],
        [ 90, 113, 158]],

       [[ 95, 117, 165],
        [ 95, 117, 165],
        [ 95, 117, 165],
        ...,
        [ 90, 113, 158],
        [ 90, 113, 158],
        [ 90, 113, 158]],

       ...,

       [[ 30,  74, 127],
        [ 30,  74, 127],
        [ 30,  74, 127],
        ...,
        [  7,  25,  54],
        [  7,  25,  54],
        [  7,  25,  54]],

       [[ 29,  73, 126],
        [ 30,  74, 127],
        [ 31,  73, 126],
        ...,
        [  7,  26,  53],
        [  7,  26,  53],
        [  7,  26,  53]],

       [[ 31,  75, 128],
        [ 31,  75, 128],
        [ 32,  74, 127],
        ...,
        [  7,  26,  53],
        [  7,  26,  53],
        [  7,  26,  53]]

In [ ]:
from flask import Flask
from flask import Response
from celery import Celery
from http import HTTPStatus
import zmq
from urllib import parse
from dotenv import load_dotenv
import argparse
import os
import json
import numpy as np
import cv2
print("Starting the socket.......")
context = zmq.Context()
    # recieve work
REQ_SOCKET_URL = os.environ.get("REQ_SOCKET_URL")
RESP_SOCKET_URL = os.environ.get("RESP_SOCKET_URL") 
consumer_receiver = context.socket(zmq.PULL)
consumer_receiver.connect(f"tcp://{REQ_SOCKET_URL}")
    # send work
consumer_sender = context.socket(zmq.PUSH)
consumer_sender.connect(f"tcp://{RESP_SOCKET_URL}")
print(f"Receiving socket url: {REQ_SOCKET_URL}")
print(f"Sender socket url: {RESP_SOCKET_URL}")
while True:
        print("rec new messageszzzzz")
        jsonData = consumer_receiver.recv_json()
        data = json.loads(jsonData)
        frame_number = data["frame_number"]
        print(frame_number)
        img_data = data["frame"]
        data["task_id"]
        print(frame_number)
        img_data = bytes.fromhex(img_data)
        img_np = cv2.imdecode(np.fromstring(img_data, dtype=np.uint8))
        print(img_np.shape)
        result = {"data": "done"}
        consumer_sender.send_json(result)



In [ ]:
# from distributed_video.load_balancer.load_balancer import LoadBalancer
# from distributed_video.load_balancer.node_manager import NodesDirectory

# nd = NodesDirectory()

# nd.close_all_sockets()
# nd.bind_all_sockets()

# lb = LoadBalancer("", nd)

